# Analyse du dataset de Plonger 

## Importation des packages

In [1]:
import numpy as np
import pandas as pd

## Resumé du dataset

Lecture du csv

In [2]:
dataset = pd.read_csv('AIDA_Results_IA_Institut.csv')

Affichage des 10 premières lignes

In [3]:
dataset.head(10)

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
5,6,Alkisti Vouraki (GRC),F,CWT,NaN,00:00,16,16 m,WHITE,16.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
6,7,Stavroula Kontorousi (GRC),F,CWT,NaN,00:00,20,20 m,WHITE,20.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
7,8,Yuliya Ilkevich (BLR),F,CWT,NaN,00:00,40,31 m,YELLOW,21.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
8,9,Dimitris Tsigkounis (GRC),M,CWT,NaN,00:00,40,26 m,YELLOW,11.0,"SHORT, TAG",Depth Event 2016,Depth Competition,2016-07-17,other
9,10,Emmanouil Marneros (GRC),M,CWT,NaN,00:00,47,47 m,WHITE,47.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other


In [4]:
dataset.shape # taille du dataset

(26841, 15)

In [5]:
dataset.isna().sum() # analyse du nombre de valeurs nan dans chaque colonnes 

Start                 0
Diver                 0
Gender                0
Discipline            0
Line              22147
Official Top          0
AP                    0
RP                    0
Card                  0
Points                0
Remarks               5
Title Event           0
Event Type            0
Day                   0
Category Event        0
dtype: int64

## Transformation des données

### Observation du nettoyage à faire 

In [6]:
dataset = dataset.drop(['Line'], axis=1)
# on retire 'Line' puique composé majoritairement de NAN

In [7]:
non_numeric_df = dataset[['Diver','Gender', 'Discipline', 'Card', 'Remarks', 'Title Event', 'Event Type', 'Day', 'Category Event']]
numeric_df = dataset[['Official Top', 'AP', 'RP', 'Points']]

In [8]:
for col in non_numeric_df.columns :
    print(col, '\n', non_numeric_df[col].value_counts(), '\n')

Diver 
 William Trubridge (NZL)    139
Georgina Miller (GBR)      135
Alexey Molchanov (RUS)     121
Alejandro Lemus (MEX)      116
Ken Kiriyama (JPN)         109
                          ... 
Yu Keunsoo (KOR)             1
Ki.Pyeong Lim (KOR)          1
Sunggwang Jung (KOR)         1
Shaun Househam (GBR)         1
Laffay Sylvain (FRA)         1
Name: Diver, Length: 4656, dtype: int64 

Gender 
 M    17434
F     9407
Name: Gender, dtype: int64 

Discipline 
 CWT     10725
FIM      8075
CNF      4813
CWTB     3228
Name: Discipline, dtype: int64 

Card 
 WHITE     19189
YELLOW     4990
RED        2524
-           138
Name: Card, dtype: int64 

Remarks 
 OK                                     8628
Ok                                     7660
-                                      2597
PEN                                    2219
No tag,rp<ap                            657
                                       ... 
DQPulling                                 1
DQ Late Start                  

on decidera d'enlever les Divers car peux interressant pour de l'IA cela pourrai biaisais, on observe plus de 1000 valeurs pour les dates ce qui ne semble pas forcément pertinent, on enlevera aussi title event car on a event type ce qui est plus intéressant on garde aussi category event

In [9]:
numeric_df 

,Official Top,AP,RP,Points
0,00:00,33,23 m,12.0
1,00:00,55,47 m,38.0
2,00:00,55,55 m,55.0
3,00:00,55,55 m,55.0
4,00:00,15,15 m,15.0
...,...,...,...,...
26836,09:40,57,57 m,57
26837,09:50,52,52 m,52
26838,10:00,40,40 m,40
26839,10:10,33,28 m,22


on enlevera official top car on ne connais pas la localisation des horraires donc sur quel fuseaux il sont de plus meme si toute les données sont enregistrées sur le meme fuseaux ce ne sont pas les memes horraires selon la localisation donc un depart à 00:00 GTM sera 01:00 à Paris et encore diffenrt à New York et transformeta RP en entier

In [10]:
# on regarde si il existe des plongeur ayant un nom prenom et pays inferieur à 10 caratère
non_numeric_df[non_numeric_df['Diver'].str.len() < 10] 

,Diver,Gender,Discipline,Card,Remarks,Title Event,Event Type,Day,Category Event
2639,(),F,CWT,WHITE,OK,AIDA Pacific Cup 2002,Mixed Competition,2002-10-31,other
4687,(),F,CWT,YELLOW,PEN,4ème Manche Coupe d AIDA France 2004,Mixed Competition,2004-05-06,other
5119,(),F,CWT,RED,DQBO,World Championship 2012,World Championship,2012-09-09,World Championship
5695,(),F,CNF,WHITE,OK,2013 Kalamata Open,Depth Competition,2013-09-09,other
5696,(),F,CNF,WHITE,OK,2013 Kalamata Open,Depth Competition,2013-09-09,other
5697,(),F,CWT,WHITE,OK,2013 Kalamata Open,Depth Competition,2013-09-09,other
7124,(),F,FIM,YELLOW,PEN,(in progress) Bizzy Blue Hole 2009,Competition,2009-04-21,other
7782,(),F,FIM,WHITE,OK,little blue hole competition,Depth Competition,2012-09-28,other
7817,(),F,CWT,WHITE,OK,AIDA Red Sea World Cup,Mixed Competition,2012-08-20,World Championship
7818,(),F,FIM,WHITE,OK,AIDA Red Sea World Cup,Mixed Competition,2012-08-20,World Championship


on observe un plongeur sans nom on decide de supprimer ces lignes car on ne sait pas si il s'agit d'un seul ou plusieur plongeurs ainsi  lors de la creation des colonnes d'experiences cela pourrais biaisais le model dans le futur

In [11]:
# on regarde si il existe des dates inferieure a la taille standard des dates
non_numeric_df[non_numeric_df['Day'].str.len() < 10 ] 

,Diver,Gender,Discipline,Card,Remarks,Title Event,Event Type,Day,Category Event


il n'y a pas de dates inferieur a la taille standart

In [12]:
numeric_df[numeric_df['RP'].str.contains('-')]

,Official Top,AP,RP,Points
2340,00:00,70,-2 m,0.0
2343,00:00,54,-2 m,0.0
2357,00:00,51,-2 m,0.0
2364,00:00,62,-2 m,0.0
2382,00:00,70,-2 m,0.0
...,...,...,...,...
25697,12:48,70,-,-
25698,12:54,35,-,-
25699,13:00,70,-,-
25700,13:06,35,-,-


On garde le RP négatif pour donner une référence d'echec au programme de machine learning. \
Cependant les tirets qui ne représentent rien sont retirés en les transformant en NAN puis drop. \
On enlevera les points donc on les transformera pas. 

### Nettoyage

In [13]:
dataset = dataset[dataset['Diver'] != '()'] # on enleve les () du dataset
dataset['RP'] = dataset['RP'].replace('-', np.nan) # on remplace les - par de NaN afin de les drops
dataset['Card'] = dataset['Card'].replace('-', np.nan) # on remplace les - par de NaN afin de les drops




In [14]:
dataset.dropna()

,Start,Diver,Gender,Discipline,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26836,7,Karine Le Flanchec (FRA),F,CWTB,09:40,57,57 m,WHITE,57,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26837,8,Anne-Sophie Passalboni (FRA),F,CWT,09:50,52,52 m,WHITE,52,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26838,9,Clementine Marie (FRA),F,CNF,10:00,40,40 m,WHITE,40,Ok,The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other
26839,10,Eisve Treciakauskaite (LTU),F,CWT,10:10,33,28 m,YELLOW,22,"No tag,under ap",The AIDA Cormorant Depth Challenge,Depth Competition,2023-07-23,other


### Transformation des données catégoriques en numérique

In [15]:
dataset['Gender'] = dataset['Gender'].replace(('M', 'F'), (1,0)) # 1 remplace Male et 0 remplace Female

In [16]:
dataset['Discipline'] = dataset['Discipline'].replace(('CNF','FIM','CWT','CWTB'),((1,2,3,4))) 
# remplacement :
#   CNF  -> 1
#   FIM  -> 2
#   CWT  -> 3
#   CWTB -> 4

### Création de caractéristiques

On récupère le mois en considérant les données de Day en string pour séparer les élément et conserver seukement la partit qui donne le mois.

In [17]:
dataset['MONTH'] = dataset['Day'].str.split('-').str[1]

In [18]:
dataset['Day'] = pd.to_datetime(dataset['Day'])

# Trier le DataFrame par date
dataset.sort_values(by='Day', inplace=True)

# Expérience totale de plongée par athlète
experience_dive = dataset.groupby('Diver')['Day'].cumcount() + 1

# Expérience totale de plongée par athlète et par discipline
experience_discipline = dataset.groupby(['Diver', 'Discipline'])['Day'].cumcount() + 1

# Ajouter les colonnes d'expérience au DataFrame d'origine
dataset['experience_dive'] = experience_dive
dataset['experience_discipline'] = experience_discipline

dataset

,Start,Diver,Gender,Discipline,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event,MONTH,experience_dive,experience_discipline
8929,1,Deborah Andollo (CUB),0,3,00:00,61,61 m,WHITE,61.0,OK,WR Attempt - ANDOLLO Deborah (CWT),Worldrecord attempt,1994-06-12,other,06,1,1
3716,1,Umberto Pelizzari (ITA),1,3,00:00,72,72 m,WHITE,72.0,OK,WR Attempt - PELIZZARI Umberto (CWT),Worldrecord attempt,1995-09-17,other,09,1,1
3713,1,Deborah Andollo (CUB),0,3,00:00,62,62 m,WHITE,62.0,OK,WR Attempt - ANDOLLO Deborah,Worldrecord attempt,1996-10-05,other,10,2,2
5021,1,Michael Oliva (FRA),1,3,00:00,72,72 m,WHITE,72.0,OK,WR Attempt - OLIVA Michael (CWT),Worldrecord attempt,1996-10-11,other,10,1,1
3717,1,Alejandro Ravelo (CUB),1,3,00:00,73,73 m,WHITE,73.0,OK,WR Attempt - RAVELO Alejandro (CWT),Worldrecord attempt,1997-08-02,other,08,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25680,2,Daham Kim (KOR),1,4,11:06,61,NaN,NaN,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship,08,6,6
25679,1,Chiara Obino (ITA),0,3,11:00,98,NaN,NaN,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship,08,18,14
25701,23,Violette Salvan-Pugliese (FRA),0,3,13:12,65,NaN,NaN,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship,08,6,6
25689,11,Matti Eronen (FIN),1,4,12:00,78,NaN,NaN,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship,08,35,31


On enlève aussi Points et Remarks car ici notre IA cherchera a predire le carton et non les points ou remarques

In [19]:
df = dataset.drop(['Start','Diver', 'Title Event', 'Official Top', 'Day', 'Points', 'Remarks'],axis=1)
df['RP'] = df['RP'].str.split(' ').str[0]
df

,Gender,Discipline,AP,RP,Card,Event Type,Category Event,MONTH,experience_dive,experience_discipline
8929,0,3,61,61,WHITE,Worldrecord attempt,other,06,1,1
3716,1,3,72,72,WHITE,Worldrecord attempt,other,09,1,1
3713,0,3,62,62,WHITE,Worldrecord attempt,other,10,2,2
5021,1,3,72,72,WHITE,Worldrecord attempt,other,10,1,1
3717,1,3,73,73,WHITE,Worldrecord attempt,other,08,1,1
...,...,...,...,...,...,...,...,...,...,...
25680,1,4,61,NaN,NaN,Depth Competition,World Championship,08,6,6
25679,0,3,98,NaN,NaN,Depth Competition,World Championship,08,18,14
25701,0,3,65,NaN,NaN,Depth Competition,World Championship,08,6,6
25689,1,4,78,NaN,NaN,Depth Competition,World Championship,08,35,31


cleanning

Sélectionner les colonnes pertinentes au problèmes (indice: les numériques, et celles
connues avant une plongée)

4. Normaliser les données pour que toutes les colonnes soient comparables en terme
de distance euclidienne

5. Appliquer deux méthodes de clustering (K-means et DBSCAN) pour détecter si
plusieurs profils de plongée se dessine :

a. sur les plongées qui ont obtenu un carton Rouge (card==RED)

b. sur les plongées qui ont obtenu un carton Blanc (card==WHITE) (pour
comprendre l'effet inverse, lorsque les plongées se passent bien)

6. Décrire et visualiser les résultats obtenus (vous pouvez afficher selon 2 ou 3
caractéristiques). Quels sont les différents clusters trouvés et leurs caractéristiques?

7. Expliquer comment vous avez choisi les hyperparamètres de chacune des méthodes

a. ￼le nombre de clusters pour K-means. Utiliser la méthode du coude, Elbow
method)

b. La densité pour DBSCAN

8. Commenter les résultats obtenus, pour et contre chaque méthode. Est-ce que les
méthodes de clustering sont adaptées au problème

Prédire si le résultat d’une plongée va être un carton blanc, jaune, rouge selon les
caractéristiques qui paraissent pertinentes au problème. Utiliser le classifieur SVM
avec différents noyaux.

2. Justifier vos choix (en phrase et par de la visualisation si nécessaire).

3. Créer un tableau récapitulant vos différents résultats.

4. Commenter les résultats obtenus.

5. Explorer d'autres pistes de classifieurs (au moins un) et comparer les résultats